In [1]:
import pandas as pd
import numpy as np

from bank_schedule.data import Data
from bank_schedule.helpers  import calc_cartesian_coords
from bank_schedule import forecast
from bank_schedule.constants import RAW_DATA_FOLDER

In [2]:
def add_last_cash_collection_date(residuals: pd.DataFrame,
                                  income: pd.DataFrame):
    """Считает ориентировочную дату последней инкассации
    на момент 2022-08-31 исходя из средней заполняемости банкомата в день

    Args:
        residuals (pd.DataFrame): _description_
        income (pd.DataFrame): _description_
    """
    avg_day_income = income.groupby('TID')['money_in'].mean()
    residuals = residuals.set_index('TID').loc[avg_day_income.index, :]
    residuals_series = residuals['money']

    days_from_last_collection = np.ceil(residuals_series / avg_day_income).astype(int)

    residuals['last_collection_date'] = residuals['date'] - pd.to_timedelta(days_from_last_collection,
                                                                            unit='D')
    return residuals.reset_index()


def get_income_on_horizon(today_date: str,
                          forecaset_model: object,
                          horizon: int=3) -> pd.DataFrame:
    """Получает суммарный приход денег в банкоматах на заданном горизонте

    Args:
        today_date (str): _description_
        forecaset_model (object): _description_
        horizon (int, optional): _description_. Defaults to 3.
    """
    predictions = forecaset_model.predict(today_date, horizon)
    return predictions.groupby('TID')['money_in'].sum().reset_index()


def get_residuals_on_horizon(today_date: str,
                             today_residuals: pd.DataFrame,
                             forecaset_model: object,
                             horizon: int=3) -> pd.DataFrame:
    """Возвращает количество денег в банкоматах через horizon дней

    Args:
        today_date (str): _description_
        today_cash (pd.DataFrame): _description_
        horizon (int, optional): _description_. Defaults to 3.
    """
    today_residuals = today_residuals.set_index('TID')['money']
    horizon_income = get_income_on_horizon(today_date, forecaset_model, horizon)
    horizon_income = horizon_income.set_index('TID').loc[today_residuals.index, 'money_in']
    horizon_residuals = today_residuals + horizon_income
    horizon_residuals.name = 'money'
    horizon_residuals = horizon_residuals.reset_index()
    horizon_residuals['date'] = pd.to_datetime(today_date) + pd.Timedelta(days=horizon)
    return horizon_residuals


def get_overflow_date(residuals: pd.DataFrame,
                      forecast_model: object,
                      overflow_thresh: int=10**6,
                      horizon: int=1,
                      max_horizon: int=100):
    """Проставляет дату переполнения

    Args:
        residuals (pd.DataFrame): текущие остатки в банкоматах
        forecast_model (object): модель прогноза
        overflow_thresh (int, optional): порог переполнения. Defaults to 10**6.
        horizon (int, optional): шаг расчета в днях. Defaults to 1.
        max_horizon (int, optional): на сколько максимум дней смотреть вперед. Defaults to 100.

    Raises:
        ValueError: _description_

    Returns:
        _type_: _description_
    """
    residuals = residuals.copy()
    if residuals['date'].unique().shape[0] != 1:
        raise ValueError('Уникальных сегодняшних дат в residuals > 1')

    today = residuals['date'].unique()[0]
    new_residuals = residuals[['TID', 'money', 'date']].copy()

    counter = 0

    while residuals['overflow_date'].isnull().any() and counter < max_horizon:
        new_residuals = get_residuals_on_horizon(today, new_residuals, forecast_model, horizon=horizon)
        overflow_cond = new_residuals['money'] > overflow_thresh
        overflow_cond &= residuals['overflow_date'].isna()

        today = today + pd.Timedelta(days=horizon)
        residuals.loc[overflow_cond, 'overflow_date'] = today
        counter += 1

    residuals['overflow_date'] = residuals['overflow_date'].fillna(today)

    return residuals

In [3]:
loader = Data(RAW_DATA_FOLDER)
dists = loader.get_distance_matrix()
geo = loader.get_geo_TIDS()
geo.set_index('TID', inplace=True)
income = loader.get_money_in()
residuals = loader.get_money_start()
residuals['date'] = pd.to_datetime('2022-08-31')
residuals = add_last_cash_collection_date(residuals, income)
residuals['overflow_date'] = pd.NaT

In [4]:
hist_mdl = forecast.ForecastHistorical()
lgbm_mdl = forecast.IncomeForecastLGBM()

In [5]:
get_overflow_date(residuals, hist_mdl)

/Users/affernus/PROJECTS/hacks/bank_schedule/src/bank_schedule/forecast.py:348: UserWarning: Дата 2022-12-01 00:00:00 не найдена в исторических данных, прогнозируем средним по TID
  warn(f'Дата {next_date} не найдена в исторических данных, прогнозируем средним по TID')
/Users/affernus/PROJECTS/hacks/bank_schedule/src/bank_schedule/forecast.py:348: UserWarning: Дата 2022-12-02 00:00:00 не найдена в исторических данных, прогнозируем средним по TID
  warn(f'Дата {next_date} не найдена в исторических данных, прогнозируем средним по TID')
/Users/affernus/PROJECTS/hacks/bank_schedule/src/bank_schedule/forecast.py:348: UserWarning: Дата 2022-12-03 00:00:00 не найдена в исторических данных, прогнозируем средним по TID
  warn(f'Дата {next_date} не найдена в исторических данных, прогнозируем средним по TID')
/Users/affernus/PROJECTS/hacks/bank_schedule/src/bank_schedule/forecast.py:348: UserWarning: Дата 2022-12-04 00:00:00 не найдена в исторических данных, прогнозируем средним по TID
  warn(f'Д

,TID,money,date,last_collection_date,overflow_date
0,406136,160000,2022-08-31,2022-08-29,2022-09-10
1,406139,387000,2022-08-31,2022-08-28,2022-09-05
2,406145,287000,2022-08-31,2022-08-28,2022-09-06
3,406148,355000,2022-08-31,2022-08-25,2022-09-10
4,406180,597000,2022-08-31,2022-08-23,2022-09-05
...,...,...,...,...,...
1625,699578,47000,2022-08-31,2022-08-26,2022-11-30
1626,699579,236000,2022-08-31,2022-08-25,2022-09-23
1627,699629,67000,2022-08-31,2022-08-29,2022-09-16
1628,699641,278000,2022-08-31,2022-08-20,2022-10-01


In [7]:
get_overflow_date(residuals, lgbm_mdl)

,TID,money,date,last_collection_date,overflow_date
0,406136,160000,2022-09-01,2022-08-29,2022-09-11
1,406139,387000,2022-09-01,2022-08-28,2022-09-06
2,406145,287000,2022-09-01,2022-08-28,2022-09-07
3,406148,355000,2022-09-01,2022-08-25,2022-09-12
4,406180,597000,2022-09-01,2022-08-23,2022-09-07
...,...,...,...,...,...
1625,699578,47000,2022-09-01,2022-08-26,2022-11-02
1626,699579,236000,2022-09-01,2022-08-25,2022-09-17
1627,699629,67000,2022-09-01,2022-08-29,2022-09-17
1628,699641,278000,2022-09-01,2022-08-20,2022-09-24
